In [1]:
import model
from utility import readCsv#

In [2]:
# u = readCsv('disorder.csv')

In [3]:
firstModel = model.IoisModel(verbose = True)

In [4]:
%prun firstModel.run_model()

['Time: ', 1, 'Apps: ', 37, 'Pay: ', 2383.009563493747, 'Cost', 2220.0, 'Actor Connections', 509, 'Application Connections', 1017]
['Time: ', 2, 'Apps: ', 37, 'Pay: ', 5669.845663157621, 'Cost', 2340.0, 'Actor Connections', 538, 'Application Connections', 2493]
['Time: ', 3, 'Apps: ', 38, 'Pay: ', 9014.691478807508, 'Cost', 2638.0, 'Actor Connections', 593, 'Application Connections', 3588]
['Time: ', 4, 'Apps: ', 38, 'Pay: ', 12342.747619782856, 'Cost', 2998.0, 'Actor Connections', 611, 'Application Connections', 3969]
['Time: ', 5, 'Apps: ', 38, 'Pay: ', 15792.694702001594, 'Cost', 3474.0, 'Actor Connections', 633, 'Application Connections', 4241]
['Time: ', 6, 'Apps: ', 38, 'Pay: ', 19199.192700487933, 'Cost', 4074.0, 'Actor Connections', 625, 'Application Connections', 4378]
['Time: ', 7, 'Apps: ', 38, 'Pay: ', 22519.531158229896, 'Cost', 4792.0, 'Actor Connections', 625, 'Application Connections', 4465]
['Time: ', 8, 'Apps: ', 38, 'Pay: ', 26040.648429372108, 'Cost', 5632.0, 'Actor

         79252745 function calls in 45.438 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     7994    8.244    0.001   21.296    0.003 agents.py:33(communicate)
   650388    7.591    0.000    7.591    0.000 utility.py:13(match)
 10878648    4.051    0.000    5.066    0.000 reportviews.py:783(__iter__)
  8536366    3.650    0.000   10.768    0.000 graph.py:452(__getitem__)
  8536366    2.768    0.000    3.566    0.000 coreviews.py:81(__getitem__)
  8536366    2.680    0.000    3.553    0.000 graph.py:338(adj)
 17072732    1.671    0.000    1.671    0.000 coreviews.py:44(__init__)
  8536366    1.327    0.000    1.327    0.000 coreviews.py:53(__getitem__)
    36205    1.210    0.000    9.948    0.000 agents.py:23(<dictcomp>)
      300    1.122    0.004   40.972    0.137 schedule.py:58(step)
   360000    0.965    0.000    0.965    0.000 {method 'binomial' of 'numpy.random.mtrand.RandomState' objects}
 10833149    0.938    0.0

In [7]:
for actor in firstModel.schedule.actorList.values():
    print(actor.tier, actor.payoffs, actor.costs)

1 3241.7930475812573 2019.0
2 3144.1132579985915 2058.0
3 3423.8526557852947 2199.0
1 2473.9894116417468 1980.0
2 3375.7727824528747 2124.0
3 3301.470048000535 2128.0
1 2804.133959715002 2199.0
2 3284.44499019489 2120.0
3 3441.5860646263623 2236.0
1 2824.9398846448016 1923.0
2 3193.8444038795033 1856.0
3 3310.40803417584 2141.0
1 2840.26002616601 2049.0
2 3066.7688965439456 2006.0
3 3297.793654587442 2042.0
1 2791.3341575901372 1977.0
2 3225.5093069528302 2113.0
3 3297.6615592770263 1939.0
1 2862.4719859652137 1886.0
2 3467.3390369457074 2173.0
3 2876.8749036015874 1867.0
1 2427.983122592325 1642.0
2 3363.7673917045017 2116.0
3 3063.43701002531 1989.0
1 2621.6177056177517 1718.0
2 3200.9890033960605 2047.0
3 3149.2000317802513 2144.0
1 2809.4788348072793 2022.0
2 3162.0547679708584 2046.0
3 3494.0807479031073 2238.0
1 2811.1794750996232 1732.0
2 3472.6723237402075 2173.0
3 3265.615758273202 2061.0
1 2995.7614862383284 1989.0
2 3078.1639509025263 2099.0
3 3140.061196436827 1942.0
1 2641

In [9]:
import time
import numpy as np

In [80]:
from mesa import Agent
import numpy as np
from utility import match
from operator import itemgetter

def communicate1(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')


def communicate2(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')

In [6]:
from utility import match
import time
import networkx as nx
import pandas as pd

In [82]:
self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time1 = 0.0
for i in range(1000):
    start1 = time.time()
    communicate1(self, schedule)
    stop1 = time.time()
    time1 += stop1 - start1

self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time2 = 0.0
for i in range(1000):
    start2 = time.time()
    communicate2(self, schedule)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

2.331467628479004
56.53080701828003


In [7]:
v1 = firstModel.schedule.appList[2544].av
v2 = [0,1,0,1,0,1,0,1,0,1]

In [8]:
def sum1(v1, v2):
    sum = 0.0
    for a,b in zip(v1, v2):
        if a=='?':
            sum += 0.5
        elif str(a)==str(b):
            sum += 1.0
    return sum

def sum2(v1, v2):
    sum = np.sum([0.5 if a == '?' else 1.0 for a, b in zip(v1, v2) if a == '?' or str(a)==str(b)])

In [12]:
time1 = 0.0
for i in range(100000):
    start1 = time.time()
    sum = sum1(v1, v2)
    stop1 = time.time()
    time1 += stop1 - start1

time2 = 0.0
for i in range(100000):
    start2 = time.time()
    sum = sum2(v1, v2)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

0.6902368068695068
1.4492120742797852


In [11]:
import time
import numpy as np
from random import choices
from random import choice

In [44]:
schedule = firstModel.schedule
self = firstModel.schedule

In [1]:
from experiment import runExperiment
from multiprocessing import Pool
import csv

In [2]:
#10 runs in 271.2s
#for range in range(50,61):
#    print(runExperiment(range))

In [3]:
#10 runs in 109.5s
#1000 runs in 
results = set()
for i in range(2):
    p = Pool(8)
    results = results.union(p.map(runExperiment, [round(x * 0.01, 2) for x in range(101)]))
    p.close()
    p.join()
    print(f'Iteration {i + 1} finished.')

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)

Iteration 1 finished.
Iteration 2 finished.


In [5]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)

In [18]:
testGraph = nx.generators.community.LFR_benchmark_graph(200, 3.0, 1.5, 0.1, average_degree=20, min_community=20)